In [2]:
### MXIMUM LIKELIHOOD ESTIMATION ###
# How to quickly implement new maximum likelihood models in statsmodels
# The GenericLikelihoodModel class esas the process by providing
# tools such as automatic numeric differenciation and an unified
# interface to scipy optimization functions. Using statsmodels
# users can fit new  MLE models simply by "plugging-in" a log-likelihood
# function

# EXAMPLE 1: PROBIT MODEL


In [4]:
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

In [5]:
# The Spector dataset is distributed with statsmodels. You can access
# values for the DEPENDENT/ENDOG variable anf a matrix of REGRESSOR/
# INDEPENDET/EXOG like this:
data = sm.datasets.spector.load_pandas()
exog = data.exog
endog = data.endog
print(sm.datasets.spector.NOTE)

::

    Number of Observations - 32

    Number of Variables - 4

    Variable name definitions::

        Grade - binary variable indicating whether or not a student's grade
                improved.  1 indicates an improvement.
        TUCE  - Test score on economics test
        PSI   - participation in program
        GPA   - Student's grade point average



In [6]:
data.exog.head()

,GPA,TUCE,PSI
0,2.66,20.0,0.0
1,2.89,22.0,0.0
2,3.28,24.0,0.0
3,2.92,12.0,0.0
4,4.00,21.0,0.0


In [7]:
data.endog.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    1.0
Name: GRADE, dtype: float64

In [8]:
# We add a constannt to the matrix of regressors
exog = sm.add_constant(exog, prepend = True)

In [9]:
# Create your own Likelihood Model: simply ovverwrite the loglike method
class MyProbit(GenericLikelihoodModel):
    def loglike(self, params):
        exog = self.exog
        endog = self.endog
        q = 2 * endog - 1
        return stats.norm.logcdf(q * np.dot(exog, params)).sum()

In [10]:
# Estimate the model and print summary
sm_probit_manual = MyProbit(endog, exog).fit()
sm_probit_manual.summary()

Optimization terminated successfully.
         Current function value: 0.400588
         Iterations: 292
         Function evaluations: 494


<class 'statsmodels.iolib.summary.Summary'>
"""
                               MyProbit Results                               
==============================================================================
Dep. Variable:                  GRADE   Log-Likelihood:                -12.819
Model:                       MyProbit   AIC:                             33.64
Method:            Maximum Likelihood   BIC:                             39.50
Date:                Tue, 17 May 2022                                         
Time:                        10:04:14                                         
No. Observations:                  32                                         
Df Residuals:                      28                                         
Df Model:                           3                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -7.4523      2.542     -2.931      0.003     -12.435      -2.469
GPA            1.6258      0.694      2.343      0.019       0.266       2.986
TUCE           0.0517      0.084      0.617      0.537      -0.113       0.216
PSI            1.4263      0.595      2.397      0.017       0.260       2.593
==============================================================================
"""

In [11]:
# Compare to stats model cannes implemantartion
sm_probit_canned = sm.Probit(endog, exog).fit()

Optimization terminated successfully.
         Current function value: 0.400588
         Iterations 6


In [12]:
print(sm_probit_canned.params)
print(sm_probit_manual.params)

const   -7.452320
GPA      1.625810
TUCE     0.051729
PSI      1.426332
dtype: float64
[-7.45233176  1.62580888  0.05172971  1.42631954]


In [13]:
print(sm_probit_canned.cov_params())
print(sm_probit_manual.cov_params())

          const       GPA      TUCE       PSI
const  6.464166 -1.169668 -0.101173 -0.594792
GPA   -1.169668  0.481473 -0.018914  0.105439
TUCE  -0.101173 -0.018914  0.007038  0.002472
PSI   -0.594792  0.105439  0.002472  0.354070
[[ 6.46416770e+00 -1.16966617e+00 -1.01173181e-01 -5.94789009e-01]
 [-1.16966617e+00  4.81472117e-01 -1.89134591e-02  1.05438228e-01]
 [-1.01173181e-01 -1.89134591e-02  7.03758403e-03  2.47189233e-03]
 [-5.94789009e-01  1.05438228e-01  2.47189233e-03  3.54069514e-01]]


# EXAMPLE 2: NEGATIVE BINOMIAL REGRESSION FOR COUNT DATA

In [16]:
# Using the nbinom distrbution from scipy we can write likelihood like this
import numpy as np
from scipy.stats import nbinom

In [17]:
def _ll_nb2(y, X, beta, alph):
    mu = np.exp(np.dot(X, beta))
    size = 1/alph
    prob = size/(size+mu)
    ll = nbinom.logpmf(y, size, prob)
    return ll

In [18]:
### NEW MODEL CLASS ###
from statsmodels.base.model import GenericLikelihoodModel

class NBin(GenericLikelihoodModel):
    def __init__(self, endog, exog, **kwds):
        super(NBin, self).__init__(endog, exog, **kwds)

    def nloglikeobs(self, params):
        alph = params[-1]
        beta = params[:-1]
        ll = _ll_nb2(self.endog, self.exog, beta, alph)
        return -ll

    def fit(self, start_params=None, maxiter=10000, maxfun=5000, **kwds):
        # we have one additional parameter and we need to add it for summary
        self.exog_names.append('alpha')
        if start_params == None:
            # Reasonable starting values
            start_params = np.append(np.zeros(self.exog.shape[1]), .5)
            # intercept
            start_params[-2] = np.log(self.endog.mean())
        return super(NBin, self).fit(start_params=start_params,
                                     maxiter=maxiter, maxfun=maxfun,
                                     **kwds)

In [19]:
# Usage example
import statsmodels.api as sm

medpar = sm.datasets.get_rdataset("medpar", "COUNT", cache=True).data
medpar.head()

,los,hmo,white,died,age80,type,type1,type2,type3,provnum
0,4,0,1,0,0,1,1,0,0,30001
1,9,1,1,0,0,1,1,0,0,30001
2,3,1,1,1,1,1,1,0,0,30001
3,9,0,1,0,0,1,1,0,0,30001
4,1,0,1,1,1,1,1,0,0,30001


In [20]:
#The model we are interested in has a vector of non-negative 
# integers as dependent variable (los), and 5 regressors:
# Intercept, type2, type3, hmo, white.

#For estimation, we need to create two variables to hold our
# regressors and the outcome variable. These can be ndarrays or
# pandas objects.
y = medpar.los
X = medpar[["type2", "type3", "hmo", "white"]].copy()
X["constant"] = 1

In [23]:
mod = NBin(y, X)
res = mod.fit()

Optimization terminated successfully.
         Current function value: 3.209014
         Iterations: 805
         Function evaluations: 1238


In [24]:
print('Parameters: ', res.params)
print('Standard errors: ', res.bse)
print('P-values: ', res.pvalues)
print('AIC: ', res.aic)

Parameters:  [ 0.2212642   0.70613942 -0.06798155 -0.12903932  2.31026565  0.44575147]
Standard errors:  [0.05059259 0.07613047 0.05326096 0.0685414  0.06794696 0.01981542]
P-values:  [1.22298084e-005 1.76979047e-020 2.01819053e-001 5.97481232e-002
 2.15207253e-253 4.62688811e-112]
AIC:  9604.95320583016


In [25]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 NBin Results                                 
==============================================================================
Dep. Variable:                    los   Log-Likelihood:                -4797.5
Model:                           NBin   AIC:                             9605.
Method:            Maximum Likelihood   BIC:                             9632.
Date:                Tue, 17 May 2022                                         
Time:                        10:20:49                                         
No. Observations:                1495                                         
Df Residuals:                    1490                                         
Df Model:                           4                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
type2          0.2213      0.051      4.373      0.000       0.122       0.320
type3          0.7061      0.076      9.275      0.000       0.557       0.855
hmo           -0.0680      0.053     -1.276      0.202      -0.172       0.036
white         -0.1290      0.069     -1.883      0.060      -0.263       0.005
constant       2.3103      0.068     34.001      0.000       2.177       2.443
alpha          0.4458      0.020     22.495      0.000       0.407       0.485
==============================================================================
"""

In [26]:
# TEstiung
res_nbin = sm.NegativeBinomial(y, X).fit(disp=0)
print(res_nbin.summary())

                     NegativeBinomial Regression Results                      
Dep. Variable:                    los   No. Observations:                 1495
Model:               NegativeBinomial   Df Residuals:                     1490
Method:                           MLE   Df Model:                            4
Date:                Tue, 17 May 2022   Pseudo R-squ.:                 0.01215
Time:                        10:21:07   Log-Likelihood:                -4797.5
converged:                       True   LL-Null:                       -4856.5
Covariance Type:            nonrobust   LLR p-value:                 1.404e-24
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
type2          0.2212      0.051      4.373      0.000       0.122       0.320
type3          0.7062      0.076      9.276      0.000       0.557       0.855
hmo           -0.0680      0.053     -1.276      0.2